# Keypoint Pipeline Tutorial

### Welcome to the keypoint pipeline tutorial! 
#### This is where you will learn how to use the code, while you wait for the user interface to be build

Lets start with importing the necessary libraries and code files

In [ ]:
# Imports
import keypoint_pipeline as kp
import grid_cut as gc
import vedo 
import time

if you found that you were not able to import these libraries and packages succesfully, you might not be using the right kernel.
Check that you are using the "pyshot_kernel" in the top right corner. Otherwise, consult the environment tutorial.

Once you have everything setup, lets begin:

### Load your teeth

You should have two stl-files that you want to compare.
Start out by clarifying where they are on your computer, by setting the paths.


In [ ]:
# Define mesh paths
AM_path = "/path/to/am_teeth.stl"
PM_path = "/path/to/pm_teeth.stl"
output_path = "/path/to/a/directory"

Now we want to load the meshes into memory

In [ ]:
# Load meshes
AM_mesh = vedo.load(AM_path)
PM_mesh = vedo.load(PM_path)

### Remove soft tissue

To remove soft tissue, we will use an updated version of the grid-cutting method. For more info, consult the grid cutting methods github page: https://github.com/AnikaKofodPetersen/Grid-Cutting/tree/main

In [ ]:
AM_cutmesh = gc.threshold_cut(AM_mesh, threshold=8.0, radius=12.0, teeth_type="max", name="AM_CUT", out_path=output_path)
PM_cutmesh = gc.threshold_cut(PM_mesh, threshold=8.0, radius=12.0, teeth_type="max", name="PM_CUT", out_path=output_path)

And make sure you work with the newest version of your meshes

In [ ]:
# Load meshes
AM_cutmesh = vedo.load(output_path+"AM_CUT.stl")
PM_cutmesh = vedo.load(output_path+"PM_CUT.stl")

### Detect keypoints

To detect keypoints, use the following function

In [ ]:
# Detect keypoints
AM_keypoints_idx, AM_keypoints_pts = kp.keypoint_detection(AM_cutmesh, name = "AM_keypoints", res=40, returnIdx = True, returnPts = True, inspection = True, output=output_path)
PM_keypoints_idx, PM_keypoints_pts = kp.keypoint_detection(PM_cutmesh, name = "PM_keypoints", res=40, returnIdx = True, returnPts = True, inspection = True, output=output_path)

To save non-encoded keypoints, use the save function

In [ ]:
# Save keypoints
kp.save_keypoints(AM_keypoints_idx, "AM_keypoints_idx", output_path)
kp.save_keypoints([p.tolist() for p in AM_keypoints_pts], "AM_keypoints_pts", output_path)
kp.save_keypoints(PM_keypoints_idx, "PM_keypoints_idx", output_path)
kp.save_keypoints([p.tolist() for p in PM_keypoints_pts], "PM_keypoints_pts", output_path)

### Represent keypoints

To SHOT-encode the keypoints, do the following:

In [ ]:
# Represent keypoints
AM_SHOT = kp.calculate_SHOT(AM_keypoints_idx,AM_cutmesh, radius=2)
PM_SHOT = kp.calculate_SHOT(PM_keypoints_idx,PM_cutmesh, radius=2)

To save the keypoints as a JSON file, use the save fucntion

In [ ]:
# Save keypoints
kp.save_keypoints(AM_SHOT, "AM_SHOT", output_path)
kp.save_keypoints(PM_SHOT, "PM_SHOT", output_path)

### Compare keypoints

To compare keypoints between the two meshes, use the compare function

In [ ]:
score = kp.keypoint_correspondence(AM_SHOT, PM_SHOT, AM_keypoints_pts, PM_keypoints_pts)
print("Final Similarity Score: ",score)

And there you have it, the final score of similarity :) 